# Introduction

In this notebook, I will explore RFM Analysis and its utility in designing personalized recommender systems

RFM stands for the three dimensions:
- Recency – How recently did the customer purchase?
- Frequency – How often do they purchase?
- Monetary Value – How much do they spend?

The resulting segments can be ordered from most valuable (highest recency, frequency, and monetary value) to least valuable (lowest recency, frequency, and value). Identifying the most valuable RFM segments can help capitalize on a good recommendation 

# Import Statements

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime

import statsmodels.api as sm

# Data: E-commerce 

The most common application for RFM analyses is in the e-commerce sector. The following e-commerce data was sourced from Kaggle.
- Data: https://www.kaggle.com/hendraherviawan/customer-segmentation-using-rfm-analysis-r/data

In [2]:
df = pd.read_csv('data.csv', encoding='unicode_escape')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


##  Data Wrangling

In [3]:
# remove incorrect entries
df = df[df['Quantity'] > 0]
# Calculate total expenditure on item
df['Monetary Value'] = df['UnitPrice'] * df['Quantity']
# Select relevant columns
df = df[['StockCode', 'Description', 'InvoiceDate', 'Monetary Value', 'CustomerID']]
# Format date
df['InvoiceDate'] = df['InvoiceDate'].apply(lambda x: x[:-5].strip())
df['InvoiceDate'] = df['InvoiceDate'].apply(lambda x: datetime.datetime.strptime(x, '%M/%d/%Y'))
df['year'] = df['InvoiceDate'].apply(lambda x: x.year)
# Select purchases from the year 2011
df = df[df['year'] == 2011]
df = df.drop(columns = ['year'])
df

,StockCode,Description,InvoiceDate,Monetary Value,CustomerID
42481,22386,JUMBO BAG PINK POLKADOT,2011-01-04 00:01:00,19.50,13313.0
42482,21499,BLUE POLKADOT WRAP,2011-01-04 00:01:00,10.50,13313.0
42483,21498,RED RETROSPOT WRAP,2011-01-04 00:01:00,10.50,13313.0
42484,22379,RECYCLING BAG RETROSPOT,2011-01-04 00:01:00,10.50,13313.0
42485,20718,RED RETROSPOT SHOPPER BAG,2011-01-04 00:01:00,12.50,13313.0
...,...,...,...,...,...
541904,22613,PACK OF 20 SPACEBOY NAPKINS,2011-01-09 00:12:00,10.20,12680.0
541905,22899,CHILDREN'S APRON DOLLY GIRL,2011-01-09 00:12:00,12.60,12680.0
541906,23254,CHILDRENS CUTLERY DOLLY GIRL,2011-01-09 00:12:00,16.60,12680.0
541907,23255,CHILDRENS CUTLERY CIRCUS PARADE,2011-01-09 00:12:00,16.60,12680.0


# RFM Calculations

### Calculate User Recency

We shall define recency as the (user item purchase frequency over the last 2 weeks) / (user item purchase frequency over the 2 weeks prior)

In [4]:
df['InvoiceDate'].max()

Timestamp('2011-01-31 00:10:00')

In [5]:
# Calculate number of weeks since last purchase in database
# TO-DO make last user purchase
df['weeks'] = df['InvoiceDate'].apply(
    lambda x: (2011 - x.year) * 52 + (5 - x.isocalendar()[1])
)
df = df[df['weeks'] > 0]
df

,StockCode,Description,InvoiceDate,Monetary Value,CustomerID,weeks
42481,22386,JUMBO BAG PINK POLKADOT,2011-01-04 00:01:00,19.50,13313.0,4
42482,21499,BLUE POLKADOT WRAP,2011-01-04 00:01:00,10.50,13313.0,4
42483,21498,RED RETROSPOT WRAP,2011-01-04 00:01:00,10.50,13313.0,4
42484,22379,RECYCLING BAG RETROSPOT,2011-01-04 00:01:00,10.50,13313.0,4
42485,20718,RED RETROSPOT SHOPPER BAG,2011-01-04 00:01:00,12.50,13313.0,4
...,...,...,...,...,...,...
541904,22613,PACK OF 20 SPACEBOY NAPKINS,2011-01-09 00:12:00,10.20,12680.0,4
541905,22899,CHILDREN'S APRON DOLLY GIRL,2011-01-09 00:12:00,12.60,12680.0,4
541906,23254,CHILDRENS CUTLERY DOLLY GIRL,2011-01-09 00:12:00,16.60,12680.0,4
541907,23255,CHILDRENS CUTLERY CIRCUS PARADE,2011-01-09 00:12:00,16.60,12680.0,4


In [6]:
# Count number of purchases by week per user per item
user_freq_grouped = df.groupby(['CustomerID', 'StockCode', 'weeks']).count()['Description'].reset_index()
user_freq_grouped.columns = ['CustomerID', 'StockCode', 'weeks', 'User Frequency (Grouped)']

In [7]:
# Count number of purchases in last 2 weeks per user per item
user_freq_2_weeks = user_freq_grouped[user_freq_grouped['weeks'].isin([0, 1, 2])]
user_freq_2_weeks = user_freq_2_weeks.groupby(['CustomerID', 'StockCode']).sum().drop(columns=['weeks']).reset_index()
user_freq_2_weeks.columns = ['CustomerID', 'StockCode', 'User Frequency 2 weeks']

In [8]:
# Count number of purchases in the 2 weeks prior per user per item
user_freq_2_weeks_prev = user_freq_grouped[user_freq_grouped['weeks'].isin([3, 4])]
user_freq_2_weeks_prev = user_freq_2_weeks_prev.groupby(['CustomerID', 'StockCode']).sum().drop(columns=['weeks']).reset_index()
user_freq_2_weeks_prev.columns = ['CustomerID', 'StockCode', 'User Frequency 2 weeks prev']

In [9]:
# Calculate recency using above definition
user_R = user_freq_2_weeks.merge(user_freq_2_weeks_prev, on=['CustomerID', 'StockCode'], how='outer')
user_R['User Frequency 2 weeks prev'] = user_R['User Frequency 2 weeks prev'].fillna(1)
user_R['R'] = user_R['User Frequency 2 weeks'] / user_R['User Frequency 2 weeks prev']
user_R = user_R.fillna(0)
user_R

,CustomerID,StockCode,User Frequency 2 weeks,User Frequency 2 weeks prev,R
0,12346.0,23166,1.0,1.0,1.0
1,12347.0,20719,1.0,1.0,1.0
2,12347.0,20966,1.0,1.0,1.0
3,12347.0,21035,1.0,1.0,1.0
4,12347.0,21041,1.0,1.0,1.0
...,...,...,...,...,...
237035,18287.0,35967,0.0,1.0,0.0
237036,18287.0,47422,0.0,1.0,0.0
237037,18287.0,72351A,0.0,1.0,0.0
237038,18287.0,72351B,0.0,1.0,0.0


In [10]:
user_R = user_R[['CustomerID', 'StockCode', 'R']]

### Calculate User Frequency
We shall define user frequency for each item as the number of times it was purchased by the user over their entire purchase history

In [11]:
user_F = df.groupby(['CustomerID', 'StockCode']).count()['Description'].reset_index()
user_F.columns=['CustomerID', 'StockCode', 'F']
user_F

,CustomerID,StockCode,F
0,12346.0,23166,1
1,12347.0,16008,1
2,12347.0,17021,1
3,12347.0,20665,1
4,12347.0,20719,2
...,...,...,...
237035,18287.0,84920,1
237036,18287.0,85039A,2
237037,18287.0,85039B,3
237038,18287.0,85040A,2


### Calculate User Monetary Value
We shall define user monetary value for each item as the total amount of money spent by the user over their entire purchase history

In [12]:
user_M = df.groupby(['CustomerID', 'StockCode']).sum('Monetary Value').reset_index()
user_M = user_M[['CustomerID', 'StockCode', 'Monetary Value']]
user_M.columns = ['CustomerID', 'StockCode', 'M']
user_M

,CustomerID,StockCode,M
0,12346.0,23166,77183.60
1,12347.0,16008,6.00
2,12347.0,17021,10.80
3,12347.0,20665,17.70
4,12347.0,20719,17.00
...,...,...,...
237035,18287.0,84920,15.00
237036,18287.0,85039A,139.20
237037,18287.0,85039B,176.40
237038,18287.0,85040A,79.20


# Merge RFM

In [13]:
RFM = user_R.merge(user_F, on=['CustomerID', 'StockCode'])
RFM = RFM.merge(user_M, on=['CustomerID', 'StockCode'])
RFM

,CustomerID,StockCode,R,F,M
0,12346.0,23166,1.0,1,77183.60
1,12347.0,20719,1.0,2,17.00
2,12347.0,20966,1.0,1,12.50
3,12347.0,21035,1.0,1,17.70
4,12347.0,21041,1.0,2,35.40
...,...,...,...,...,...
237035,18287.0,35967,0.0,1,13.68
237036,18287.0,47422,0.0,1,10.08
237037,18287.0,72351A,0.0,1,50.40
237038,18287.0,72351B,0.0,1,50.40


# Create RFM Score using CDFs
To generate a recommendation, we will need to create a score from the RFM metrics, giving them equal weightage. To do this, we shall use the cumulative distribution function (CDF) for each of R, F, and M and obtain a probability from 0 to 1 for each metric. We use this probability to group the recency, frequency, and monetary value into three buckets (from high to low). These ranks are then summed to generate a score for each item for each user

In [14]:
def return_edges(cdf):
    '''
    This function returns the edges by which to segment recency, frequency, 
    and monetary value into three buckets using the probabilistic properties 
    of the CDF
    '''
    bin_edges = []
    bin_edges.append(cdf.x[0])

    for lst in np.array_split(cdf.x, 3):
        bin_edges.append(lst[len(lst) - 1])
        
    return bin_edges

def return_groups(val, bin_edges, inverse=False):
    '''
    This function uses the edges from the return_edges function 
    to output a rank (from 1 to 3) for recency, frequency, and monetary value
    '''
    if val >= bin_edges[0] and val <= bin_edges[1]:
        if inverse:
            return 3
        else:
            return 1
    elif val >= bin_edges[1] and val <= bin_edges[2]:
        return 2
    elif val >= bin_edges[2] and val <= bin_edges[3]:
        if inverse:
            return 1
        else:
            return 3
    else:
        return None

In [15]:
# Generate CDFS
R_cdf = sm.distributions.ECDF(RFM['R'])
F_cdf = sm.distributions.ECDF(RFM['F'])
M_cdf = sm.distributions.ECDF(RFM['M'])

# Generate Bins
R_edges = return_edges(R_cdf)
F_edges = return_edges(F_cdf)
M_edges = return_edges(M_cdf)

# Assign labels to groups 
R_groups = RFM['R'].apply(lambda x: return_groups(x, R_edges))
F_groups = RFM['F'].apply(lambda x: return_groups(x, F_edges))
M_groups = RFM['M'].apply(lambda x: return_groups(x, M_edges))

# Create new columns 
RFM_sum = RFM.assign(
    R_Group = R_groups, 
    F_Group = F_groups, 
    M_Group = M_groups, 
)
RFM_sum = RFM_sum[['CustomerID', 'StockCode', 'R_Group', 'F_Group', 'M_Group']]

# Create Final Score by summing RFM ranks
RFM_sum['Final Score'] = RFM_sum[['R_Group', 'F_Group', 'M_Group']].sum(axis = 1)
RFM_sum = RFM_sum[RFM_sum['Final Score'] > 0]

In [16]:
# Add item descriptions
RFM_sum = RFM_sum.merge(df[['CustomerID', 'StockCode', 'Description']], on=['CustomerID', 'StockCode'], how='left')
RFM_sum = RFM_sum.drop_duplicates()

## Inspect Sorted Recommendations for a Particular Customer

In [17]:
RFM_sum[RFM_sum['CustomerID'] == 12347.0].sort_values(['Final Score'], ascending=False)

,CustomerID,StockCode,R_Group,F_Group,M_Group,Final Score,Description
24,12347.0,22423,2,3,3,8,REGENCY CAKESTAND 3 TIER
38,12347.0,84625A,2,3,3,8,PINK NEW BAROQUECANDLESTICK CANDLE
5,12347.0,21041,2,3,3,8,RED RETROSPOT OVEN GLOVE DOUBLE
36,12347.0,84558A,2,3,3,8,3D DOG PICTURE PLAYING CARDS
8,12347.0,21731,2,3,3,8,RED TOADSTOOL LED NIGHT LIGHT
...,...,...,...,...,...,...,...
222001,12347.0,20665,1,1,2,4,RED RETROSPOT PURSE
222015,12347.0,22821,1,1,1,3,GIFT BAG PSYCHEDELIC APPLES
222030,12347.0,23506,1,1,1,3,MINI PLAYING CARDS SPACEBOY
222031,12347.0,23508,1,1,1,3,MINI PLAYING CARDS DOLLY GIRL


In [18]:
# TO-DO: What do we do when the final score is the same?
# TO-DO: Incorporate RFM for entire user base as well as individual user preferences 